In [ ]:
# huggingface-cli login

In [3]:
# =======================
# Setup (一度だけ実行)
# =======================
# pip install -U "transformers>=4.41" "sentence-transformers>=3.0.0" "huggingface_hub>=0.23" torch

import os
from typing import List
import numpy as np
from numpy.linalg import norm

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# ---- 認証（事前に https://huggingface.co/settings/tokens で Read トークン発行）----
HF_TOKEN = os.getenv("HUGGINGFACE_HUB_TOKEN")  # 既に環境変数があれば使う
if HF_TOKEN:
    login(token=HF_TOKEN)

# =======================
# ① Gemma 3n-E4B (生成) の比較
# =======================
GEN_MODEL_ID = "google/gemma-3n-E4B"  # 生成モデル（Gemma 3n）
gen_tok = AutoTokenizer.from_pretrained(GEN_MODEL_ID, use_auth_token=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
    use_auth_token=True,
)

def generate_answer(prompt: str, max_new_tokens: int = 128) -> str:
    inputs = gen_tok(prompt, return_tensors="pt").to(gen_model.device)
    out = gen_model.generate(**inputs, max_new_tokens=max_new_tokens)
    return gen_tok.decode(out[0], skip_special_tokens=True)

# =======================
# ② EmbeddingGemma (検索/類似度) の比較
# =======================
EMB_MODEL_ID = "google/embeddinggemma-300m"  # 埋め込みモデル
emb_model = SentenceTransformer(EMB_MODEL_ID, use_auth_token=True)

def rank_by_similarity(query: str, docs: List[str]):
    q = emb_model.encode_query(query)              # shape: (768,)
    d = emb_model.encode_document(docs)            # shape: (N, 768)
    # SentenceTransformersのsimilarityでもOKだが、ここではcos類似度で明示
    sims = (q @ d.T) / (norm(q) * norm(d, axis=1)) # shape: (N,)
    order = np.argsort(-sims)                      # 降順ソート
    return sims, order

# =======================
# デモ用データ（日本語）
# =======================
query = "赤い惑星として知られている惑星はどれですか？"
documents = [
    "金星は地球の双子と呼ばれることがあります。",
    "火星は赤っぽい外観で知られ、赤い惑星と呼ばれています。",
    "木星は太陽系で最大の惑星で、大赤斑があります。",
    "土星はその美しい環で有名ですが、赤い惑星と間違われることもあります。"
]

# =======================
# 実行
# =======================
print("=== ① Gemma 3n-E4B（生成） ===")
gen_response = generate_answer(query)
print(gen_response)
print()

print("=== ② EmbeddingGemma（検索/類似度） ===")
sims, order = rank_by_similarity(query, documents)
for idx in order:
    print(f"{sims[idx]:.3f} | {documents[idx]}")

# 期待挙動：
# ① 生成：テキストで「火星です」などの答えを直接生成
# ② 検索：類似度が最も高い文として「火星...赤い惑星...」が上位に来る


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/haraguchi/projects/EmbeddingGemma-exp/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 3077.10 MB. The target location /Users/haraguchi/.cache/huggingface/hub/models--google--gemma-3n-E4B/blobs only has 2909.79 MB free disk space.
  warnings.warn(
/Users/haraguchi/projects/EmbeddingGemma-exp/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 4966.79 MB. The target location /Users/haraguchi/.cache/huggingface/hub/models--google--gemma-3n-E4B/blobs only has 2909.79 MB free disk space.
  warnings.warn(
/Users/haraguchi/projects/EmbeddingGemma-exp/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 4992.87 MB. The target location /Users/haraguchi/.cache/h

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

RuntimeError: Data processing error: CAS service error : IO Error: No space left on device (os error 28)